# Image inversion with numpy

## Read training data

In [1]:
import numpy as np
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt

In [15]:
# Convert PIL image to 200x200 array normalised to -0.5 to 0.5 rage.
def image_2_array(filename):
    # PIL image pil_img
    pil_img=Image.open(filename)
    pil_arr=np.array(pil_img)
    pil_max = np.amax(np.absolute(pil_arr))
    pil_min = np.amin(np.absolute(pil_arr))
    #print ("Max/min:"+ str(pil_max)+"/"+str(pil_min))
    if pil_max == 0:
        return pil_img,pil_arr
    pil_scaled_arr = (pil_arr / pil_max) - 0.5
    return pil_img,pil_scaled_arr

# Invert normalised PIL image
def invert_array(arr):
    #print(arr.shape)
    #print(arr[100][50:60])
    inverted_arr = arr * (-1)
    #print(inverted_arr[100][50:60])
    return inverted_arr
    
# Convert normalized array to PIL image
def array_2_image(arr):
    MAX = 255
    arr_scaled = (arr + 0.5) * MAX
    arr_max = np.amax(arr_scaled)
    arr_min = np.amin(arr_scaled)
    #print ("Array Max/min:"+ str(arr_max)+"/"+str(arr_min))
    im = Image.fromarray(arr_scaled)
    return im

# I. Image inversion

In [16]:
# Prepare inversed images
plot = False
save = False  # Only need to save once
if save:
    for dataset in range(1, 11):
        for img_counter in range(1, 11):
            # Read Image
            dataset_str = str(dataset).zfill(3)
            image_str = str(img_counter).zfill(4)
            filename = "/notebooks/data001/move" + \
                    dataset_str+"_"+image_str+".gif"
            #print filename
            img,a = image_2_array(filename)
            #print("Orgarr:")
            #print(a)


            in_arr = invert_array(a)
            #print("Invarr:")
            #print(in_arr)
            new_im = array_2_image(in_arr)
            new_filename = "/notebooks/data001/move_inv_" + \
                    dataset_str+"_"+image_str+".gif"
            # Save new image to file
            new_im.save(new_filename, "GIF")
            # Plot
            if (plot):
                _, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4)
                ax1.imshow(img);
                ax2.imshow(a, vmin=-0.5, vmax=0.5,cmap=plt.cm.Greys);
                ax3.imshow(in_arr, cmap=plt.cm.Greys);
                ax4.imshow(new_im);



# Prepare training set

In [17]:
# Read training images into array of shape (80, 200, 200, 1)
IMSIZE=200
train_data_onedimension = np.array([]) #np.empty([80,200,200,1])
train_labl_onedimension = np.array([])
for dataset in range(1, 9):
    for img_counter in range(1, 11):
        dataset_str = str(dataset).zfill(3)
        image_str = str(img_counter).zfill(4)
        # Samples
        filename = "/notebooks/data001/move" + \
                dataset_str+"_"+image_str+".gif"
        print filename
        img,a = image_2_array(filename)
        
        train_data_onedimension = np.append(train_data_onedimension,a)
        # Targets (labels)
        filename = "/notebooks/data001/move_inv_" + \
                dataset_str+"_"+image_str+".gif"
        print filename
        img,a = image_2_array(filename)
        train_labl_onedimension = np.append(train_labl_onedimension,a)
        
        
print train_data_onedimension.shape
print train_labl_onedimension.shape

images_in_set = train_data_onedimension.size / (IMSIZE*IMSIZE)
print images_in_set
train_data = train_data_onedimension.reshape(images_in_set, IMSIZE, IMSIZE, 1)
print train_data.shape
train_labl = train_labl_onedimension.reshape(images_in_set, IMSIZE, IMSIZE, 1)
print train_labl.shape

/notebooks/data001/move001_0001.gif
/notebooks/data001/move_inv_001_0001.gif
/notebooks/data001/move001_0002.gif
/notebooks/data001/move_inv_001_0002.gif
/notebooks/data001/move001_0003.gif
/notebooks/data001/move_inv_001_0003.gif
/notebooks/data001/move001_0004.gif
/notebooks/data001/move_inv_001_0004.gif
/notebooks/data001/move001_0005.gif
/notebooks/data001/move_inv_001_0005.gif
/notebooks/data001/move001_0006.gif
/notebooks/data001/move_inv_001_0006.gif
/notebooks/data001/move001_0007.gif
/notebooks/data001/move_inv_001_0007.gif
/notebooks/data001/move001_0008.gif
/notebooks/data001/move_inv_001_0008.gif
/notebooks/data001/move001_0009.gif
/notebooks/data001/move_inv_001_0009.gif
/notebooks/data001/move001_0010.gif
/notebooks/data001/move_inv_001_0010.gif
/notebooks/data001/move002_0001.gif
/notebooks/data001/move_inv_002_0001.gif
/notebooks/data001/move002_0002.gif
/notebooks/data001/move_inv_002_0002.gif
/notebooks/data001/move002_0003.gif
/notebooks/data001/move_inv_002_0003.gif

## Newral Network 

L – Number of layers = 3

$s_j$ – кол-во элементов в слое j

Кол-во элементов по слоям:
$s_1$ = 40000
$s_2$ = 40000
$s_3$ = 40000



In [18]:
L = 3
s = np.ones(L, dtype=np.int) * IMSIZE*IMSIZE
s

array([40000, 40000, 40000])

### 1. Random $\Theta$ initialisation

$\epsilon=0.5$

In [19]:
INIT_EPSILON=0.5
Theta = []
for j in range(1,L):
    print(j)
    print(s[j-1])
    theta = np.random.rand(s[j],s[j-1]+1) * (2*INIT_EPSILON) - INIT_EPSILON
    Theta.append(theta) 
print(Theta)

1
40000
2
40000
[array([[ 0.04019491,  0.00877941,  0.28164199, ..., -0.29980837,
        -0.05458793, -0.13429882],
       [ 0.27307652,  0.19263834,  0.45375294, ...,  0.08290362,
        -0.33900697,  0.22474366],
       [-0.40393647,  0.20432185,  0.0694239 , ...,  0.07500261,
        -0.21728321, -0.37293107],
       ..., 
       [ 0.37316968,  0.03458426, -0.41070868, ...,  0.356509  ,
         0.37365916, -0.35365806],
       [ 0.29708414, -0.09842149, -0.36000103, ..., -0.08217825,
         0.39726725,  0.28294943],
       [-0.20899501, -0.42017115,  0.36603819, ..., -0.00825868,
        -0.11170189,  0.29392853]]), array([[-0.44497826,  0.10997881, -0.3227901 , ...,  0.14463931,
         0.34160487, -0.45184907],
       [-0.11444854,  0.06083013, -0.11166088, ..., -0.23592035,
         0.12838341,  0.37345527],
       [ 0.2840222 ,  0.22242093, -0.06866587, ..., -0.1150945 ,
        -0.07154383, -0.42332406],
       ..., 
       [ 0.00730122,  0.37400741,  0.17107083, ...,  0.

_Note: Access $\Theta_1$ as `Theta[0]`, $\Theta_2$ as `Theta[1]`._


For regularisation computations use matrix $\Theta$ without first (bias) column. Store it in list of arrays `Theta_r`.

### 2. Forward propagation

In [20]:
# Return cost and gradient for given Theta.
# Parameters:
# Theta – weights,
# L – number of layers in NN,
# s – number of elements in each layer,
# X – training dataset,
# Y – target dataset,
# lmbd – regularisation parameter lambda.
def costFunction(Theta, L, s, X, Y, lmbd):
    # Initialisation. Need to return the following variables:
    J = 0;
    Theta_grad = []
    for j in range(1,L):
        Theta_grad.append(np.zeros(size(Theta[j-1]))
                          
    # Remove first column from Theta1
    Theta_r = []
    for j in range(1,L):
        theta_reduced = Theta[j-1][:, 1:]
        Theta_r.append(theta_reduced)
        print(Theta_r[j-1].shape)


SyntaxError: invalid syntax (<ipython-input-20-a71ee79f0505>, line 16)

In [27]:
x = np.array([[[1],[2],[3]], [[4],[5],[6]]])
x.shape
x[:,0:2]

SyntaxError: invalid syntax (<ipython-input-27-943645028040>, line 3)